# Tarefa 4: interface conversacional, chat os LLMs Nova Lite e Titan Embeddings

Neste caderno, você desenvolve um chatbot utilizando os modelos Foundation Nova Lite e Titan Embeddings (FMs) disponíveis no Amazon Bedrock.

Interfaces conversacionais, como chatbots e assistentes virtuais, podem aprimorar a experiência do usuário para seus clientes. Os chatbots usam algoritmos de processamento de linguagem natural (PLN) e machine learning para entender e responder às consultas dos usuários. Você pode usar chatbots em diversas aplicações, como atendimento ao cliente, vendas e comércio eletrônico, para oferecer respostas rápidas e eficientes aos usuários. Os usuários podem acessá-los por vários canais, como sites, plataformas de redes sociais e aplicativos de mensagens.

- **Chatbot (básico)**, chatbot zero-shot com modelo FM
- **Chatbot com uso de prompt**, template(LangChain): chatbot com algum contexto fornecido no modelo de prompt
- **Chatbot com persona**: chatbot com funções definidas, por exemplo, coach de carreira e interações humanas
- **Chatbot com consciência contextual**, inclusão de contexto por meio de um arquivo externo gerando incorporações.

## Framework LangChain para criação de chatbot com o Amazon Bedrock

Em interfaces conversacionais, como chatbots, lembrar as interações anteriores é muito importante, tanto no curto quanto no longo prazo.

O framework LangChain fornece componentes de memória em duas formas. Primeiro, o LangChain oferece recursos auxiliares para gerenciar e manipular mensagens de chat anteriores. Eles são projetados para serem modulares. Em segundo lugar, o LangChain fornece maneiras fáceis de incorporar esses utilitários em cadeias, permitindo que você defina e interaja facilmente com diferentes tipos de abstrações, o que facilita a criação de chatbots avançados.

## Criar chatbot com contexto: elementos principais

O primeiro processo na criação de um chatbot com consciência contextual é gerar incorporações para o contexto. Normalmente, ocorre um processo de ingestão no seu modelo para gerar as incorporações, que depois são inseridas no armazenamento de vetores. Neste caderno, você usa o modelo do Titan Embeddings para isso. O segundo processo é a orquestração, interação, invocação e o retorno dos resultados das solicitações dos usuários. Isso envolve orquestrar a solicitação do usuário, interagir com os modelos/componentes necessários para coletar informações, invocar o chatbot para formular uma resposta e, em seguida, retornar a resposta do chatbot ao usuário.

## Tarefa 4.1: Configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [5]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


### Utilitários de otimização de recursos

Nesta seção, criamos um decorador que otimiza o uso de recursos da API e melhora a confiabilidade das solicitações. Esse decorador implementa várias práticas recomendadas para trabalhar com serviços de IA baseados em nuvem

In [6]:
import time
import random
from functools import wraps

# Resource optimization decorator for API efficiency
def optimize_resource_usage(max_attempts=10, base_pause=10.0):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    # Space out requests for optimal service performance
                    if attempt > 0:
                        time.sleep(10.0)
                    return func(*args, **kwargs)
                except Exception as e:
                    error_str = str(e)
                    # Check if this is a service capacity exception
                    if any(err in error_str for err in ["ThrottlingException", "TooManyRequests", 
                                                     "Too many tokens", "Rate exceeded"]):
                        if attempt < max_attempts - 1:
                            # Implement progressive backoff with jitter
                            jitter = random.random() * 0.5
                            wait_time = min(base_pause * (2 ** attempt) + jitter, 60)
                            
                            print(f"⏱️ Optimizing request timing. Pausing for {wait_time:.1f}s (attempt {attempt+1}/{max_attempts})")
                            time.sleep(wait_time)
                        else:
                            print("⚠️ Maximum attempts reached. Consider spacing out operations.")
                            raise
                    else:
                        raise
        return wrapper
    return decorator

### Para entender a otimização de recursos

O código acima implementa uma abordagem inteligente para o gerenciamento de APIs que:

- **Espaça as solicitações** para manter a produtividade ideal
- **Tenta de novo automaticamente** quando os limites de capacidade temporários são atingidos
- **Implementa o recuo exponencial** aumentando progressivamente os tempos de espera
- **Adiciona jitter aleatório** para evitar o agrupamento de solicitações

Essas técnicas são práticas padrão do setor para trabalhar com serviços de IA baseados em nuvem e garantem que seu aplicativo permaneça responsivo mesmo sob condições desafiadoras.

### Classes aprimoradas de modelos Bedrock

Em seguida, criamos versões especializadas das classes do modelo Bedrock que incorporam automaticamente as técnicas de otimização de recursos. Essas classes estendem as implementações padrão do LangChain e, ao mesmo tempo, adicionam recursos de confiabilidade que são essenciais para aplicativos de produção.

Ao agrupar os métodos críticos da API com nosso decorador de otimização, garantimos um comportamento consistente ao nos comunicarmos com os serviços Bedrock.

In [7]:
from langchain_aws import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings

# Enhanced ChatBedrock with resource optimization
class ResourceOptimizedChatBedrock(ChatBedrock):
    """Wrapper that optimizes resource usage and handles service capacity constraints."""
    
    @optimize_resource_usage(max_attempts=10)
    def _call_model(self, *args, **kwargs):
        return super()._call_model(*args, **kwargs)
    
    @optimize_resource_usage(max_attempts=10)
    def _generate(self, *args, **kwargs):
        return super()._generate(*args, **kwargs)
        
# Enhanced BedrockEmbeddings with resource optimization
class ResourceOptimizedEmbeddings(BedrockEmbeddings):
    """Wrapper for BedrockEmbeddings with intelligent resource management."""
    
    @optimize_resource_usage(max_attempts=10)
    def _embed_documents(self, texts):
        return super()._embed_documents(texts)
        
    @optimize_resource_usage(max_attempts=10)
    def _embed_query(self, text):
        return super()._embed_query(text)
        
    @optimize_resource_usage(max_attempts=10)
    def embed_documents(self, texts):
        return super().embed_documents(texts)
        
    @optimize_resource_usage(max_attempts=10)
    def embed_query(self, text):
        return super().embed_query(text)



<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** essas classes aprimoradas:

- Aplicam a otimização de recursos a todos os métodos críticos da API
- Mantêm total compatibilidade com as interfaces padrão do LangChain
- Lidam automaticamente com problemas intermitentes de capacidade de serviço
- Fornecem um comportamento consistente nas operações de bate-papo e incorporação

## Construir memória conversacional

Uma das características mais importantes de um chatbot eficaz é a capacidade de lembrar interações anteriores. Sem memória, cada mensagem seria tratada isoladamente, criando uma experiência desarticulada.

Nesta seção, implementamos a memória conversacional usando a classe `InMemoryChatMessageHistory` LangChain. Isso permite que nosso chatbot faça referência a trocas anteriores e mantenha o contexto durante toda a conversa.

### Utilitários de formatação de conversas

Esta seção implementa uma função utilitária que formata corretamente as conversas multifuncionais para grandes modelos de linguagem. O formato está alinhado aos requisitos específicos do formato de token do modelo Nova Lite.

In [8]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Tarefa 4.2: usar o histórico do chat do LangChain para iniciar a conversa

Nesta tarefa, você ativará o chatbot para que guarde o contexto das conversas entre as várias interações com os usuários. Ter uma memória conversacional é imprescindível para que os chatbots mantenham diálogos significativos e coerentes ao longo do tempo.

Você implementa recursos de memória conversacional com base na classe InMemoryChatMessageHistory do LangChain. Esse objeto armazena as conversas entre o usuário e o chatbot, e o histórico fica disponível para que o agente do chatbot possa aproveitar o contexto de uma conversa anterior.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** as saídas do modelo não são determinísticas.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model = ResourceOptimizedChatBedrock(
    model_id="amazon.nova-lite-v1:0", 
    client=bedrock_client
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### Novas perguntas

O modelo respondeu com uma mensagem inicial. Agora, você faz algumas perguntas.

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Aprimorar as perguntas

Agora, faça uma pergunta sem mencionar a palavra jardim para ver se a modelo consegue entender a conversa anterior.

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Encerrar a conversa

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## Tarefa 4.3: Chatbot usando modelo de prompt (LangChain)

### Uso de modelos de prompt para interações consistentes

A engenharia de prompts é um aspecto essencial do trabalho com grandes modelos de linguagem. Usando modelos, podemos estruturar nossos prompts de forma consistente e controlar como o modelo responde em diferentes cenários.

Os modelos de prompt do LangChain fornecem uma maneira padronizada de formatar entradas para diferentes funções de conversa (sistema, usuário, assistente), garantindo que o modelo receba um contexto adequadamente estruturado.

Nesta tarefa, você usa o PromptTemplate padrão que é responsável pela construção dessa entrada. O LangChain oferece várias classes e funções para facilitar a criação e trabalho com prompts.

In [13]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [14]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Em seguida, inicie uma conversa.

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## Tarefa 4.4: Chatbot com persona

Nesta tarefa, o assistente de Inteligência Artificial (IA) desempenha a função de coach de carreira. Você pode usar uma mensagem do sistema para informar ao chatbot qual persona (ou função) ele vai desempenhar. Continue aproveitando a classe InMemoryChatMessageHistory para manter o contexto da conversa.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

Agora, faça uma pergunta que não esteja dentro da especialidade dessa persona. O modelo não deve responder a essa pergunta e deve dar uma razão para isso.

## Tarefa 4.5 Chatbot com contexto

Nesta tarefa, você pedirá que o chatbot responda perguntas com base no contexto que foi passado para ele. Você usa um arquivo CSV e o modelo do Titan Embeddings para criar um vetor representando esse contexto. Esse vetor é armazenado no Facebook AI Similarity Search (FAISS). Quando uma pergunta for feita ao chatbot, você transmitirá esse vetor de volta para o chatbot e fará com que ele recupere a resposta usando o vetor.

### Geração aumentada de recuperação (RAG)

Os chatbots tradicionais estão limitados ao conhecimento contido em seus dados de treinamento. O RAG supera essa limitação:

1. **Criando incorporações vetoriais** de seus documentos personalizados
2. **Armazenando esses vetores** em um banco de dados pesquisável (como o FAISS)
3. **Recuperando o contexto relevante** ao responder perguntas
4. **Aumento do prompt do LLM** com esse contexto adicional

Essa abordagem permite que o chatbot aproveite seu conhecimento treinado e suas fontes de informações específicas ao gerar respostas.

### Modelo do Titan Embeddings

As incorporações representam palavras, frases ou qualquer outro item distinto como vetores em um espaço de vetores contínuo. Assim, os modelos de machine learning podem fazer operações matemáticas nessas representações e capturar as relações semânticas entre elas.

Você usa incorporações para a geração aumentada de recuperação (RAG) [recurso de pesquisa de documentos](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [37]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = ResourceOptimizedEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock_client
)

### FAISS como VectorStore

Para usar incorporações na pesquisa, você precisa de um armazenamento que faça pesquisas por similaridade de vetores de forma eficaz. Neste caderno, você usa o FAISS, que é um armazenamento em memória. Para armazenar vetores permanentemente, você pode usar as Bases de Conhecimento para Amazon Bedrock, pgVector, Pinecone, Weaviate ou Chroma.

As APIs VectorStore do LangChain estão disponíveis [aqui](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### Fazer um teste rápido com pouco código 

Você pode usar uma classe wrapper fornecida pelo LangChain para consultar o armazenamento do banco de dados de vetores e retornar os documentos relevantes. Isso processa uma cadeia de QA com todos os valores padrão.

In [ ]:
chat_llm = ResourceOptimizedChatBedrock(
    model_id="amazon.nova-lite-v1:0",
    client=bedrock_client
)

# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### Aplicação de chatbot

Para o chatbot, você precisa de gerenciamento de contexto, histórico, armazenamentos de vetores e muitos outros componentes. Comece criando uma cadeia de Geração aumentada de recuperação (RAG) com suporte a contexto.

Ela usa as funções **create_stuff_documents_chain** e **create_retrieval_chain**

### Parâmetros e funções usados para a RAG

- **Recuperador:** você usou `VectorStoreRetriever`, que é apoiado por um `VectorStore`. Para recuperar texto, há dois tipos de pesquisa para escolher: `"similarity"` ou `"mmr"`. `search_type="similarity"` usa a pesquisa por similaridade no objeto recuperador, onde seleciona vetores de fragmentos de texto que são mais semelhantes ao vetor da pergunta.

- **create_stuff_documents_chain** especifica como o contexto recuperado é alimentado em um prompt e no LLM. Os documentos recuperados são "preenchidos" como contexto sem qualquer resumo ou outro processamento no prompt.

- **create_retrieval_chain** adiciona a etapa de recuperação, propaga o contexto recuperado pela cadeia e o fornece junto com a resposta final. 

Se a pergunta feita estiver fora do escopo do contexto, o modelo responderá que não sabe a resposta.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

Em seguida, inicie uma conversa

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


Você utilizou os LLMs Nova Lite e Titan para desenvolver uma interface conversacional seguindo os seguintes padrões:

- Chatbot (básico: sem contexto)
- Chatbot usando modelo de prompt (LangChain)
- Chatbot com personas
- Chatbot com contexto

### Experimente você mesmo

- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 5**.